In [1]:
from tensorflow import keras

In [2]:
from keras import Sequential
from keras.layers import Dense, Flatten

Using TensorFlow backend.


In [3]:
import joblib
import numpy as np

##### DATA

In [4]:
with open(r'F:\ОБУЧЕНИЕ\CHAT-BOTs\тексты\dialogues\_phrases2.txt', encoding='utf-8') as f:
    phrases = f.readlines()
phrases = [i.replace('\n', '') for i in phrases]
with open(r'F:\ОБУЧЕНИЕ\CHAT-BOTs\тексты\dialogues\_intents2.txt', encoding='utf-8') as f:
    intents = f.readlines()    
intents = [i.replace('\n', '') for i in intents]

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(ngram_range=(1,3))
X=tfidf.fit_transform(phrases)

In [6]:
X

<539x171 sparse matrix of type '<class 'numpy.float64'>'
	with 2178 stored elements in Compressed Sparse Row format>

In [7]:
len(tfidf.vocabulary_)

171

In [8]:
classes = list(set(intents))
y = [classes.index(i) for i in intents]

In [9]:
y[:10]

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

##### split to train/test samples

In [10]:
from keras.utils import to_categorical
y_binary = to_categorical(y)

In [11]:
y_binary

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=444)

In [13]:
x_train.shape

(431, 171)

##### MODEL

In [14]:
model = Sequential()

In [15]:
model.add(Dense(32, activation='relu', input_shape=(len(tfidf.vocabulary_),)))

In [16]:
model.add(Dense(len(classes), activation='softmax'))

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                5504      
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 231       
Total params: 5,735
Trainable params: 5,735
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(x_train
          ,y_train
          ,batch_size=10
          ,epochs=10
          ,verbose=2 
#           ,validation_split=0.2 
          ,validation_data=(x_test, y_test)
          ,shuffle=True
         )

Train on 431 samples, validate on 108 samples
Epoch 1/10
 - 1s - loss: 1.9113 - accuracy: 0.2668 - val_loss: 1.8603 - val_accuracy: 0.4352
Epoch 2/10
 - 0s - loss: 1.7954 - accuracy: 0.6334 - val_loss: 1.7451 - val_accuracy: 0.7222
Epoch 3/10
 - 0s - loss: 1.6563 - accuracy: 0.8445 - val_loss: 1.5876 - val_accuracy: 0.9907
Epoch 4/10
 - 0s - loss: 1.4626 - accuracy: 0.9884 - val_loss: 1.3667 - val_accuracy: 1.0000
Epoch 5/10
 - 0s - loss: 1.2102 - accuracy: 1.0000 - val_loss: 1.1011 - val_accuracy: 1.0000
Epoch 6/10
 - 0s - loss: 0.9302 - accuracy: 1.0000 - val_loss: 0.8209 - val_accuracy: 1.0000
Epoch 7/10
 - 0s - loss: 0.6656 - accuracy: 1.0000 - val_loss: 0.5814 - val_accuracy: 1.0000
Epoch 8/10
 - 0s - loss: 0.4575 - accuracy: 1.0000 - val_loss: 0.4039 - val_accuracy: 1.0000
Epoch 9/10
 - 0s - loss: 0.3117 - accuracy: 1.0000 - val_loss: 0.2805 - val_accuracy: 1.0000
Epoch 10/10
 - 0s - loss: 0.2157 - accuracy: 1.0000 - val_loss: 0.1990 - val_accuracy: 1.0000


In [20]:
model.evaluate(x_train,
              y_train,
              batch_size=10, 
              verbose=1)

431/431 [==============================] - 0s 291us/step


[0.1785302617848611, 1.0]

In [21]:
classes[np.argmax(model.predict(tfidf.transform(['какая погода блин?'])))]

'узнать метеопрогноз на сегодня'